In [ ]:
import numpy as np
import pandas as pd
from obspy import read
import matplotlib.pyplot as plt
import os
from scipy.signal import butter, lfilter
import features as ft
from scipy.signal import freqz
import datetime
from scipy.stats import norm
import re

In [ ]:
# Set the working directory to the location of the data files
os.chdir("C:\\Users\\javie\\OneDrive - INSTITUTO TECNOLOGICO AUTONOMO DE MEXICO\\MaestriaEnCienciaDeDatos\\EstanciaDeInvestigacion\\Popocatepelt\\PopocatepetlVolcano\\data")
# os.chdir("D:/Popocatepetl/data/clean_data/2023")

In [ ]:
# Path to the directory containing the data files
# paths = os.listdir("D:/Popocatepetl/data/clean_data/2023")
paths = os.listdir("C:\\Users\\javie\\OneDrive - INSTITUTO TECNOLOGICO AUTONOMO DE MEXICO\\MaestriaEnCienciaDeDatos\\EstanciaDeInvestigacion\\Popocatepelt\\PopocatepetlVolcano\\data") 

In [ ]:
# Filter files for a specific month
month = 'CN_PPPP_HHZ_2023_05'
with_s = [x for x in paths if re.match(r'^{}'.format(month), x)]
print(with_s)

In [ ]:
def calculate_freq_index(trace, trace_high, window_size_seconds=600, overlap=0.5):
    # Get sampling rate and calculate window size in samples
    sampling_rate = trace.stats.sampling_rate
    window_samples = int(window_size_seconds * sampling_rate)
    
    # Calculate step size (distance between window starts)
    step_samples = int(window_samples * (1 - overlap))
    
    # Get the signal data
    signal = trace.data
    signal_high = trace_high.data
    
    # Initialize lists to store results
    timestamps = []
    freq_index_values = []
    
    # Slide the window through the signal
    for i in range(0, len(signal) - window_samples + 1, step_samples):
        # Extract window
        window = signal[i:i + window_samples]
        window_high = signal_high[i:i + window_samples]
        
        # Calculate timestamp for the center of the window
        window_center_sample = i + window_samples // 2
        timestamp = trace.stats.starttime + window_center_sample / sampling_rate

        # Calculate frequency index: logarithmic ratio of high-frequency to low-frequency energy
        freq_index_value = ft.freq_index(window, window_high, window_samples)
        
        # Store results
        timestamps.append(timestamp.datetime)
        freq_index_values.append(freq_index_value)
    
    # Create DataFrame with results
    results_df = pd.DataFrame({
        'timestamp': timestamps,
        'frequency_index': freq_index_values,
    })
    
    return results_df

In [ ]:
fi_3_df = pd.DataFrame(columns=["timestamp", "frecuency_index"])
for file in with_s:
    print(file)
    [filtered_signal, s_low, s_high] = ft.load_and_preprocess_seismic_data(file, freqmin=0.7, freqmax=30, low_freqmin=1, low_freqmax=5, high_freqmin=5, high_freqmax=10)
    day_df = calculate_freq_index(s_low[0], s_high[0], window_size_seconds=600, overlap=0.5)
    fi_3_df = pd.concat([fi_3_df, day_df], ignore_index=True)

In [ ]:
plt.plot(fi_2_df['timestamp'], fi_2_df['frequency_index'], color='tab:green')
plt.title('Frequency Index (1-4 Hz and 4-9 Hz)')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
# Plot the frequency index for different frequency bands with the same y-axis range
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(14, 12), sharex=True)

# Find global min and max for all frequency index values
all_freq_index = pd.concat([
    features_df['frequency_index'],
    fi_df['frequency_index'],
    fi_3_df['frequency_index'],
    fi_2_df['frequency_index']
])
ymin, ymax = all_freq_index.min(), all_freq_index.max()

axes[0].plot(features_df['timestamp'], features_df['frequency_index'], color='tab:green')
axes[0].set_title('Frequency Index (1-5.5 Hz and 6-16 Hz)')
axes[0].set_ylim(ymin, ymax)

axes[1].plot(fi_df['timestamp'], fi_df['frequency_index'], color='tab:blue')
axes[1].set_title('Frequency Index (1-5 Hz and 5-15 Hz)')
axes[1].set_ylim(ymin, ymax)

axes[2].plot(fi_3_df['timestamp'], fi_3_df['frequency_index'], color='tab:red')
axes[2].set_title('Frequency Index (1-5 Hz and 5-10 Hz)')
axes[2].set_ylim(ymin, ymax)

axes[3].plot(fi_2_df['timestamp'], fi_2_df['frequency_index'], color='tab:orange')
axes[3].set_title('Frequency Index (1-4 Hz and 4-9 Hz)')
axes[3].set_xlabel('Time')
axes[3].set_ylim(ymin, ymax)

plt.xticks(rotation=45)